#### Create features and write it as csv file

#### Fill/Back fill motion

In [17]:
import pandas as pd
import numpy as np
import math

df = pd.read_excel("worker4_fill_backfill_jan13.xlsx", sep=',')
df.head()

,date,time,OrientPitch,OrientYaw,OrientRoll,NormAccelX,NormAccelY,NormAccelZ,NormGyroX,NormGyroY,NormGyroZ,label
0,2021-01-13,12:34:10,1.15880,1.92559,-0.91036,0.30361,-0.26033,0.91654,0.04538,-0.02094,0.04421,Filling/Backfilling
1,2021-01-13,12:34:10,1.15886,1.92977,-0.90934,0.30462,-0.26196,0.91574,0.03956,-0.02211,0.05236,Filling/Backfilling
2,2021-01-13,12:34:10,1.15902,1.93434,-0.90806,0.30362,-0.25547,0.91790,0.04072,-0.02560,0.04072,Filling/Backfilling
3,2021-01-13,12:34:10,1.15924,1.93874,-0.90707,0.30033,-0.25928,0.91791,0.04072,-0.03142,0.06283,Filling/Backfilling
4,2021-01-13,12:34:10,1.15940,1.94212,-0.90675,0.29635,-0.26360,0.91798,0.03607,-0.02211,0.06516,Filling/Backfilling


In [18]:
df.drop(columns=['date','time'],inplace=True)

Separate data into 10 data per row

In [22]:
# Transpose the data and arrange 10 data into per row, make it as a function

def separation(df):
    ori_dict = {}
    colname= ["OrientPitch", "OrientYaw", "OrientRoll",
          "NormAccelX", "NormAccelY", "NormAccelZ ",
          "NormGyroX", "NormGyroY", "NormGyroZ"]
    for col in colname:
        # Create a new list to record the data
        old = []
        new = []
        old = df[col] # Record data in each column

        row_num = math.floor(len(old)/10) # Find 
        leftover = len(old) % 10 # find the leftover of data

        # Remove the last few leftover data
        old_clear = old[: len(old) - leftover] 

        # reshape the matrix
        new = np.reshape(np.array(old_clear), (row_num, 10))

        # dataframe each feature of the data
        ori_dict[col] = pd.DataFrame(new)
    
    return ori_dict

ori_dict = separation(df)

In [23]:
# Have a look at the data in dictionary, 10 data as a row
ori_dict

{'NormAccelX':             0        1        2        3        4        5        6        7  \
 0     0.30361  0.30462  0.30362  0.30033  0.29635  0.29887  0.29607  0.29891   
 1     0.29608  0.29711  0.29979  0.29904  0.29656  0.29651  0.29468  0.29135   
 2     0.29557  0.29545  0.29406  0.29585  0.29420  0.29025  0.29255  0.29649   
 3     0.30209  0.29823  0.29252  0.29707  0.29998  0.29977  0.29750  0.29994   
 4     0.30292  0.30307  0.30094  0.29825  0.30295  0.30431  0.30492  0.30531   
 5     0.30311  0.29894  0.29652  0.30130  0.30702  0.30731  0.30798  0.30487   
 6     0.30040  0.30141  0.29857  0.29624  0.29892  0.30539  0.30718  0.30599   
 7     0.29874  0.29732  0.29840  0.30154  0.30011  0.30066  0.29848  0.29987   
 8     0.30347  0.30182  0.29710  0.29547  0.29792  0.29988  0.29862  0.29887   
 9     0.29730  0.29836  0.29825  0.29719  0.29850  0.30096  0.30292  0.29939   
 10    0.29886  0.30089  0.29951  0.29671  0.30161  0.30322  0.30414  0.29905   
 11    0.29830

In [24]:
# Calculate the mean of each feature(acce, gyro, degree) and other features

fill_backfill = ori_dict

def featurecalculation(ori_dict):
    colname= ["OrientPitch", "OrientYaw", "OrientRoll",
          "NormAccelX", "NormAccelY", "NormAccelZ ",
          "NormGyroX", "NormGyroY", "NormGyroZ"]
    feature = pd.DataFrame([])
    # 1> we calculate their mean
    for col in colname:
        name = "mean_" + col
        feature[name] = ori_dict[col].mean(axis = 1)
    
    # 2>  we calculate their standard deviation
    for col in colname:
        name = "std_" + col
        feature[name] = ori_dict[col].std(axis = 1)
        
    # 3>  we calculate their variance
    for col in colname:
        name = "var_" + col
        feature[name] = ori_dict[col].var(axis = 1)
    
    # 4>  we calculate their maximum value
    for col in colname:
        name = "max_" + col
        feature[name] = ori_dict[col].max(axis = 1)
        
    # 5>  we calculate their minimum value
    for col in colname:
        name = "min_" + col
        feature[name] = ori_dict[col].min(axis = 1)
        
    # 6>  we calculate their medium value
    for col in colname:
        name = "med_" + col
        feature[name] = ori_dict[col].median(axis = 1)
        
    return feature
        

feature = featurecalculation(fill_backfill)

In [26]:
feature = featurecalculation(ori_dict)
feature['label'] = 'fill/backfill'
feature.to_csv('worker4_fill_backfill_jan13_feature.csv')

Similarly, we can use the functions we created to organize features for other motion data

#### Get on/off the construction equipment

In [27]:
df_equipment = pd.read_excel("worker4_on_off_equipment_jan13.xlsx", sep=',') # Read data
on_off_equipment = separation(df_equipment) # Separate data, 10 data in a row
feature_equipment = featurecalculation(fill_backfill) # Create features for the dataset
feature_equipment['label'] = 'on/off equipment'
feature_equipment.to_csv('worker4_on_off_equipment_jan13_feature.csv') # Write the dataset to a csv file

#### Operating/Driving the equipment

In [28]:
df_operating = pd.read_excel("worker4_sitting_driving_operating_jan13.xlsx", sep=',')
sitting_driving_operating = separation(df_operating)
feature_operating = featurecalculation(sitting_driving_operating)
feature_operating['label'] = 'sitting/driving/operating'
feature_operating.to_csv('worker4_sitting_driving_operating_jan13_feature.csv')

#### Walking

In [29]:
df_walking = pd.read_excel("worker4_walking_jan13.xlsx", sep=',')
walking = separation(df_walking)
feature_walking = featurecalculation(walking)
feature_walking['label'] = 'walking'
feature_walking.to_csv('worker4_walking_jan13_feature.csv')

#### Compacting

In [30]:
df_compacting = pd.read_excel("worker4_compacting_jan13.xlsx", sep=',')
compacting = separation(df_compacting)
feature_compacting = featurecalculation(compacting)
feature_compacting['label'] = 'compacting'
feature_compacting.to_csv('worker4_compacting_jan13_feature.csv')

#### Guiding

In [31]:
df_guiding = pd.read_excel("worker4_guiding_jan13.xlsx", sep=',')
guiding = separation(df_guiding)
feature_guiding = featurecalculation(guiding)
feature_guiding['label'] = 'guiding'
feature_guiding.to_csv('worker4_guiding_jan13_feature.csv')

#### Loading/unloading

In [33]:
df_loading = pd.read_excel("worker4_load_unload_jan13.xlsx", sep=',')
loading = separation(df_loading)
feature_loading  = featurecalculation(loading)
feature_loading['label'] = 'loading/unloading'
feature_loading.to_csv('worker4_load_unload_jan13_feature.csv')

#### Standing/Communicating

In [34]:
df_stand = pd.read_excel("worker4_stand_communicate_jan13.xlsx", sep=',')
stand = separation(df_stand)
feature_stand  = featurecalculation(stand)
feature_stand['label'] = 'standing/communicating'
feature_stand.to_csv('worker4_stand_communicate_jan13_feature.csv')

#### Sweeping

In [35]:
df_sweep = pd.read_excel("worker4_sweep_jan13.xlsx", sep=',')
sweep = separation(df_sweep)
feature_sweep  = featurecalculation(sweep)
feature_sweep['label'] = 'sweeping'
feature_sweep.to_csv('worker4_sweep_jan13_feature.csv')